<div style="font-size: 32px; color: gray; font-weight: bold; text-align: center;">Get Data for Stock Analysis</div>

---

## Libraries

In [3]:
# !pip install yfinance

In [4]:
# Imports
import yfinance as yf
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
# import plotly
# import plotly.express as px
# import plotly.graph_objects as go
# pd.options.plotting.backend = "plotly"
import numpy as np
import boto3
from io import StringIO
from sagemaker import get_execution_role
import warnings
warnings.filterwarnings("ignore")
import pickle

Matplotlib is building the font cache; this may take a moment.


## <span style="color: red">Parameters</span>

In [5]:
# Update the following
# cob_date = "2021-05-14" # use this as the row index *** NOT USED?? ***
prev_yr_end = "2020-12-31"

# Details to capture from info function for metadata and ratio calculations:

metadata = [
    "longName",
    "sector",
    "industry",
]
    
static_ratios = [
    # ratios:
#     "trailingPE",
#     "forwardPE",
#     "priceToSalesTrailing12Months",
    "profitMargins",
    "payoutRatio",
    "sharesOutstanding",
    "trailingEps",
    "forwardEps",
]

financials = [
    "Total Revenue",
]

cashflow = [
    "Total Cash From Operating Activities",
]

balsheet = [
    "Total Liab",
    "Total Stockholder Equity",
    "Total Current Assets",
    "Total Current Liabilities",
]

# Type translation
# cob_date = pd.to_datetime(cob_date) #.date() # is this the right date format?
prev_yr_end = pd.to_datetime(prev_yr_end) #.date() # is this the right date format?

In [6]:
# print(cob_date, type(cob_date))
print(prev_yr_end, type(prev_yr_end))

2020-12-31 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


## Investment Ratios

### Price-earnings ratio

Divide a company's share price by its annual earnings per share to calculate the P/E ratio. This ratio shows how much investors are willing to pay for $1 of a company's earnings. "It is probably the best way of comparing assets in different sectors and of finding true bargains," says Steven Jon Kaplan, CEO of True Contrarian Investments. Higher P/E ratios suggest a company's future earnings are expected to grow and may appear overvalued compared with companies with lower P/Es. That said, a high or low P/E doesn't necessarily indicate a good or bad investment; it offers a snapshot that begs additional inquiry. Given the uncertainty of future cash flows, it can be helpful for investors to rely on historical P/Es and use a mix of other ratios to evaluate and pick stocks.

Calc:

- Adj Close / trailingEps
- Adj Close / forwardEps

NB: can compare across industries/sectors

### Price-sales ratio

Robert Johnson, professor of finance at Creighton University in Omaha, Nebraska, touts the benefits of calculating a stock's price divided by sales per share, commonly referred to as the price-to-sales ratio. "The price-to-sales ratio is used by analysts who want to eliminate some of the distortions that can result in company earnings," Johnson says. It's a useful ratio to determine whether a company has earnings, cash flow or even positive book value since sales is always a positive number. A lower ratio suggests you've found a bargain, or a value stock. Industry consensus says lower P/S stocks have better value because investors are paying less for every dollar of a company's revenues. P/S ratio values can vary across sectors, so to best assess a company's P/S, compare it with industry peers.

In [7]:
# Calc:

# a. Total Revenue

# b. sharesOutstanding

# c. Adj Close / (a/b)

# NB: compare to peers in same industry

### Profit margin ratio

This is the amount of profit a company makes for every unit of sales. Investors calculate this ratio by dividing net profit over revenue. Profit margins are unique to an industry – with grocery chains known for low profit margins, while software companies can claim double-digit ratios. But this information doesn't necessarily mean that it's better to buy a software company than a grocery store stock. A high profit margin means a business can offer products priced higher than its costs, yielding profits through effective pricing strategies. A low profit margin may mean there are inefficient pricing strategies, where a business cannot produce enough profit to cover expenses. Any stock could be a winner with a growing revenue stream and steady profit margins.

Calc:
- profitMargins

NB: compare to peers in same industry. Low profit margin could be ok if revenue growing and profit margin steady.

### Dividend payout ratio

Companies with rising dividend payments are favored by John Robinson, owner and founder of Nest Egg Guru in Hawaii. The dividend payout ratio is the percentage of net income paid to investors in the form of dividends. This ratio tells investors how much earnings are paid out in dividends versus how much is reinvested back into the company. The higher the percentage, the less money remains to reinvest back into growing the company. "Companies that pay out less than 60% of their earnings as dividends tend to have room for further dividend increases and the ability to withstand temporary earnings downturns without having to reduce or eliminate dividend payments," he says.

Calc:
- payoutRatio

NB: amount of earnings paid out as dividend. Rest is retained / reinvested. Lower % may indicate growth or ability to withstand earnings shocks.

### Price-free cash flow ratio

Tim Parker, a partner at Regency Wealth Management in New York City, reveres free cash flow because that is the amount of money left over after a company reinvests in the business to pay dividends, buy back shares or make acquisitions. To determine price-free cash flow, divide the company's share price by the operating free cash flow per share. The ratio measures how much cash a company earns for each share of stock. Investors want to search for companies with growing free cash flow that are selling at a bargain. Parker favors this ratio since free cash flow is harder to manipulate than earnings. A lower ratio indicates a company may be undervalued, while a higher ratio may signal overvaluation.

In [8]:
# Calc:

# 1. Operating cashflow for previous year end:
# su = yf.Ticker("SU.PA")
# su.cashflow[pd.to_datetime('2020-12-31')].loc["Total Cash From Operating Activities"]
# PLUS
# su.cashflow[pd.to_datetime('2020-12-31')].loc["Capital Expenditures"]

# 2. Divided by
# sharesOutstanding 

# 3. Equals: Operating Free Cashflow per Share

# 4. Finally: Adj Close / Operating Free Cashflow per Share

# NB: this ratio focusses on cash rather than earnings (cash is harder to manipulate). 
# A lower ratio may indicate company undervalued and high ratio may indicate overvalued.


### Debit-equity ratio

Valuation ratios are important, but so are quality measures, such as debt and liquidity metrics. Divide a company's total liabilities by its shareholder equity to compute the debt-equity ratio. This ratio explains a company's financial leverage, the comparison between borrowed funds and equity or ownership. Think of this ratio like a homeowner's mortgage value versus principal on the home. A greater proportion of debt constrains a company's flexibility to grow as more revenue is directed to pay debt costs. Like most ratios, compare the debt-equity ratio to those of other industry members, as some sectors, such as utilities, have higher typical debt ratios compared with others.

In [81]:
# Calc:
# su = yf.Ticker("SU.PA")
# tot_liab = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Liab"]
# tot_se = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Stockholder Equity"]
# de_ratio = tot_liab/tot_se
# print(tot_liab)
# print(tot_se)
# print(de_ratio)

# NB: high ratio (leverage) may indicate has less flexibility to grow as it is constrained by its debt costs.
# Compare to other companies in same industry as different sectors have different typical leverage. 
# SEEMS LIKE THERE ARE MULTIPLE WAYS TO IMPLEMENT. CONSISTENCY IS MOST IMPORTANT.

### Current ratio

Sameer Samana, global equity and technical strategist at Wells Fargo Investment Institute in St. Louis, recommends examining the quick ratio and current ratio. These liquidity ratios measure if a company has enough working capital to handle potential downturns and financial setbacks. The current ratio divides current assets by current liabilities to measure how much cash a company has on hand to pay short-term obligations within a year. The quick ratio sums cash, marketable securities and accounts receivables and divides this sum by current liabilities. Higher numbers for these ratios suggest greater liquidity, while lower ratios may suggest a company cannot meet short-term obligations.

In [88]:
# Calc:
# su = yf.Ticker("SU.PA")
# ca = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Current Assets"]
# cl = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Current Liabilities"]
# current_ratio = ca/cl
# print(ca)
# print(cl)
# print(current_ratio)

# NB: working capital indicates ability to handle potential financial downturns and setbacks.

### EBITDA-to-sales ratio

This metric is the company's EBITDA – which is an abbreviation for earnings before interest, taxes, depreciation and amortization – divided by its net sales. This ratio is used to evaluate a company's overall profitability or earnings before expenses, by comparing revenue with earnings. "The stability of EBITDA typically determines investors' appetite for the amount of debt it believes the business should have," says Bryan Lee, chief investment officer at Blue Zone Wealth Advisors in Los Angeles. "A more levered company brings higher volatility for how the equity trades. This volatility can amplify returns on the upside but also to the downside," he says. EBITDA margin offers a transparent view into business operations by eliminating noncash or nonoperating expenses like interest costs, taxes and depreciation that may dim profits, giving a more precise view of a company's profitability.

In [11]:
# Calc:

# su = yf.Ticker("SU.PA")
# su.financials
# su.cashflow[pd.to_datetime('2020-12-31')].loc["Total Cash From Operating Activities"]

# NB: don't see how to get EBITDA from YF

## Functions

In [12]:
# Function to save a .csv in S3 from a DataFrame of our SageMaker python notebook
def _write_dataframe_to_csv_on_s3(DESTINATION, dataframe, filename):
    """ Write a dataframe to a CSV on S3 """
    print(
        "Writing {} records to '{}' in bucket '{}'.".format(
            len(dataframe), filename, DESTINATION
        )
    )
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=",", index=True)
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(DESTINATION, filename).put(Body=csv_buffer.getvalue())

## Fetch data

### Get ticker info

In [13]:
mystock = "SU.PA" # ticker code per Yahoo Finance
myticker = yf.Ticker(mystock)

In [14]:
info_ticker = myticker.info
for info in info_ticker:
    print("-" * 72)
    print(info, ":")
    print(info_ticker[info])

------------------------------------------------------------------------
zip :
92500
------------------------------------------------------------------------
sector :
Industrials
------------------------------------------------------------------------
fullTimeEmployees :
135000
------------------------------------------------------------------------
longBusinessSummary :
Schneider Electric S.E. provides energy and automation digital solutions worldwide. The company operates in two segments, Energy Management and Industrial Automation. It offers busway and cable management products, circuit breakers and switches, contactors and protection relays, electrical protection and control products, fuse and safety switches, motor starters and protection components, power monitoring and control products, power quality and power factor correction products, pushbuttons, switches, pilot lights and joysticks, software products, surge protection and power conditioning products, and switchboards and en

In [15]:
# for key in myticker.info:
#     print(key)

# print(myticker.info["longName"])
# print(myticker.info["sector"])
# print(myticker.info["industry"])

In [16]:
# mytickers = yf.Tickers(["MC.PA", "RDSA.AS"])

In [17]:
# for ticker in mytickers.tickers:
#     print(mytickers.tickers[ticker].info["longName"])

### Parameters for the history function

### Get history of a stock

In [18]:
# hist = myticker.history(period="1y") # check docs for parameters

In [19]:
# hist.head()

In [20]:
# hist["Close"].plot(figsize=(16,9), grid=True, title=f"{mystock}\n{myticker.info['longName']}") # without Plotly backend

### Fetch historical prices, volumes etc.

In [21]:
# Stocks comprising the euronext 100
euronext100 = ['KBC.BR', 'JMT.LS', 'BNP.PA', 'BN.PA', 'GBLB.BR', 'KN.PA', 'AGS.BR', 'LR.PA', 'MC.PA', 'ABI.BR', 'EDEN.PA', 'RDSA.AS', 'OR.PA', 'CAP.PA', 'SAN.PA', 'GALP.LS', 'SW.PA', 'INGA.AS', 'KER.PA', 'PHIA.AS', 'ATO.PA', 'ILD.PA', 'COV.PA', 'AIR.PA', 'AC.PA', 'TEP.PA', 'UBI.PA', 'ASML.AS', 'RAND.AS', 'VIE.PA']

# All stocks on Euronext Paris, Amsterdam & Brussels
euronext_pab = ['2CRSI.PA', '2MX.PA', 'ASP.PA', 'AALB.AS', 'ABI.BR', 'AB.PA', 'ABCA.PA', 'ABEO.PA', 'ABNX.PA', 'ABVX.PA', 'ABN.AS', 'ABO.BR', 'ACPH.BR', 'ACAN.PA', 'ACCEL.AS', 'ACCB.BR', 'AC.PA', 'ACNV.PA', 'AXS.AS', 'ACKB.BR', 'EOS.PA', 'ATI.PA', 'ALDV.PA', 'ALP.PA', 'ADOC.PA', 'ADP.PA', 'ADUX.PA', 'ADV.PA', 'ADVIC.PA', 'ADVI.PA', 'ADYEN.AS', 'AED.BR', 'AGN.AS', 'AKOM.PA', 'AGS.BR', 'AGFB.BR', 'AD.AS', 'AF.PA', 'AI.PA', 'AIR.PA', 'AJAX.AS', 'AKA.PA', 'AKW.PA', 'AKZA.AS', 'AAA.PA', 'ABIO.PA', 'ALD.PA', 'ALFEN.AS', 'CDANV.PA', 'CDA.PA', 'ALM.PA', 'ALO.PA', 'ALONV.PA', 'LTA.PA', 'ALTA.PA', 'AREIT.PA', 'ATE.PA', 'ALTUR.PA', 'ALX.AS', 'AMG.AS', 'AMPLI.PA', 'ACOMO.AS', 'AMUN.PA', 'APAM.AS', 'APM.PA', 'ARCAD.AS', 'MT.AS', 'ARG.PA', 'ARGX.BR', 'AKE.PA', 'ARTE.PA', 'PRC.PA', 'ARTO.PA', 'ASC.BR', 'ASIT.BR', 'ASM.AS', 'ASML.AS', 'ASRNL.AS', 'ASY.PA', 'ATA.PA', 'ATEME.PA', 'ATEB.BR', 'ATO.PA', 'ATRS.AS', 'AUB.PA', 'AUGR.PA', 'AURE.PA', 'AURS.PA', 'AVTX.AS', 'AVT.PA', 'CS.PA', 'CSNV.PA', 'AXW.PA', 'BSGR.AS', 'BCRA.PA', 'BAIN.PA', 'BALTA.BR', 'BALYO.PA', 'BAMNB.AS', 'BANI.BR', 'BUI.PA', 'BAR.BR', 'BFIT.AS', 'BASS.PA', 'BLC.PA', 'BESI.AS', 'BEFB.BR', 'BEKB.BR', 'FBEL.PA', 'BELR.BR', 'BELU.BR', 'BEN.PA', 'BBED.AS', 'BEVER.AS', 'BB.PA', 'BIG.PA', 'BCART.BR', 'BIM.PA', 'BLEE.PA', 'BNP.PA', 'BOI.PA', 'BOL.PA', 'BOLNV.PA', 'BON.PA', 'BOTHE.BR', 'BOOST.PA', 'BOKA.AS', 'BSD.PA', 'BGHL.AS', 'BGHS.AS', 'EN.PA', 'ENNV.PA', 'BPOST.BR', 'BNB.BR', 'BREB.BR', 'BRILL.AS', 'BRNL.AS', 'BVI.PA', 'BUR.PA', 'CAT31.PA', 'CAFO.PA', 'CBDG.PA', 'CAMB.BR', 'CAPLI.PA', 'CAP.PA', 'CPINV.BR', 'CARM.PA', 'CARP.PA', 'CA.PA', 'CO.PA', 'CAS.PA', 'CATG.PA', 'CTRG.PA', 'CATR.PA', 'CBOT.PA', 'CGM.PA', 'CGR.PA', 'CYAD.BR', 'CENER.BR', 'CFEB.BR', 'CFI.PA', 'CGG.PA', 'CRI.PA', 'CHSR.PA', 'CDI.PA', 'CIB.PA', 'COMB.BR', 'CLA.PA', 'CMCOM.AS', 'COM.PA', 'CNV.PA', 'CNP.PA', 'CCEP.AS', 'COFA.PA', 'COFB.BR', 'COH.PA', 'RE.PA', 'COLR.BR', 'CRBN.AS', 'CLB.AS', 'COUR.PA', 'COV.PA', 'COVH.PA', 'CRAP.PA', 'CRAV.PA', 'CRBP2.PA', 'CIV.PA', 'CRLA.PA', 'CRLO.PA', 'CMO.PA', 'CNF.PA', 'CCN.PA', 'CAF.PA', 'CRSU.PA', 'CRTO.PA', 'ACA.PA', 'OPTI.BR', 'CROS.PA', 'SX.PA', 'CTAC.AS', 'CTPNV.AS', 'CLEX.BR', 'DIE.BR', 'BN.PA', 'AM.PA', 'DSYNV.PA', 'DSY.PA', 'DBV.PA', 'DECB.BR', 'DLTA.PA', 'DBG.PA', 'DVT.PA', 'DGB.AS', 'DGE.PA', 'DGM.PA', 'DPAM.PA', 'DPA.AS', 'DSC2.AS', 'DSC2S.AS', 'DSM.AS', 'EAS2P.AS', 'ECONB.BR', 'EDEN.PA', 'EDF.PA', 'EFIC1.AS', 'EFICT.AS', 'GID.PA', 'FGR.PA', 'EKI.PA', 'EKOP.BR', 'ELEC.PA', 'EEM.PA', 'ELI.BR', 'ELIOR.PA', 'ELIS.PA', 'ENGI.PA', 'EPS.PA', 'ENVI.AS', 'EOSI.PA', 'ERA.PA', 'ERYP.PA', 'ESG.AS', 'ESGT.AS', 'ESI.PA', 'ESP.AS', 'ELNV.PA', 'EL.PA', 'ES.PA', 'EFI.PA', 'RF.PA', 'EUR.PA', 'ECT.AS', 'ECMPA.AS', 'ERFNV.PA', 'ERF.PA', 'EURN.BR', 'ENX.PA', 'ECP.PA', 'EUCAR.PA', 'ETL.PA', 'EGR.PA', 'EVS.BR', 'EXAC.PA', 'EXE.PA', 'EXM.BR', 'EPCP.PA', 'EXPL.PA', 'FAGR.BR', 'FAST.AS', 'EO.PA', 'FAUV.PA', 'FAYE.PA', 'FDJ.PA', 'FCMC.PA', 'FALG.PA', 'SACI.PA', 'ORIA.PA', 'FGA.PA', 'BERR.PA', 'EBPF.PA', 'FOAF.PA', 'FINM.PA', 'ODET.PA', 'FNTS.PA', 'FIPP.PA', 'FLOB.BR', 'FLOW.AS', 'FLUX.BR', 'FNAC.PA', 'FNG.BR', 'FPN.PA', 'LEBL.PA', 'FATL.PA', 'EURS.PA', 'INEA.PA', 'FLY.PA', 'SPEL.PA', 'FORE.PA', 'FFARM.AS', 'FOU.BR', 'LFDE.PA', 'FREY.PA', 'FUR.AS', 'GLPG.AS', 'GALIM.PA', 'GAM.PA', 'GBLB.BR', 'GEA.PA', 'GFC.PA', 'GNE.PA', 'GENX.PA', 'GNRO.PA', 'GNFT.PA', 'GKTX.PA', 'GV.PA', 'SIGHT.PA', 'GOJXN.AS', 'GET.PA', 'GIMB.BR', 'GLO.PA', 'GLOG.BR', 'GPE.PA', 'GRVO.PA', 'GVNV.AS', 'GREEN.BR', 'CEN.PA', 'FLO.PA', 'GOE.PA', 'IRD.PA', 'GJAJ.PA', 'OPN.PA', 'PARP.PA', 'SFPI.PA', 'GTBP.PA', 'GTT.PA', 'GBT.PA', 'GBTNV.PA', 'GUI.PA', 'HAL.AS', 'HAMO.BR', 'PIG.PA', 'HEIJM.AS', 'HEIA.AS', 'HEIO.AS', 'RMS.PA', 'HEXA.PA', 'HF.PA', 'HCO.PA', 'HIPAY.PA', 'HOLCO.AS', 'HOMI.BR', 'HOP.PA', 'HDP.PA', 'HDG.AS', 'HUNDP.AS', 'HYDRA.AS', 'HYL.BR', 'IBAB.BR', 'ICAD.PA', 'ICT.AS', 'IDL.PA', 'IDIP.PA', 'IEP.BR', 'IEX.AS', 'IGE.PA', 'ILD.PA', 'IMCD.AS', 'NK.PA', 'IMMOU.BR', 'IMDA.PA', 'IMMO.BR', 'INCLU.BR', 'INFE.PA', 'INF.PA', 'INGA.AS', 'IPH.PA', 'INPST.AS', 'ITP.PA', 'INTER.AS', 'INTO.BR', 'ITXT.PA', 'IVA.PA', 'IPN.PA', 'IPS.PA', 'IPSNV.PA', 'ITE.PA', 'JBOG.PA', 'JCQ.PA', 'DEC.PA', 'JDEP.AS', 'JEN.BR', 'TKWY.AS', 'KOF.PA', 'KBC.BR', 'KBCA.BR', 'KENDR.AS', 'KER.PA', 'KEYW.BR', 'KDS.AS', 'KIN.BR', 'LI.PA', 'KORI.PA', 'KPN.AS', 'OR.PA', 'ORNV.PA', 'LACR.PA', 'LHN.PA', 'MMB.PA', 'LAT.PA', 'LPE.PA', 'LVIDE.AS', 'LOUP.PA', 'LEAS.BR', 'LBON.PA', 'LSS.PA', 'LSSNV.PA', 'LR.PA', 'LIN.PA', 'FII.PA', 'LNA.PA', 'LOTB.BR', 'BOLS.AS', 'LBIRD.PA', 'MC.PA', 'LYS.PA', 'MRM.PA', 'MDM.PA', 'MALT.PA', 'MTU.PA', 'MAN.PA', 'MAREL.AS', 'IAM.PA', 'MKEA.PA', 'MAU.PA', 'MBWS.PA', 'MCPHY.PA', 'MDXH.BR', 'EDI.PA', 'MEDCL.PA', 'MELE.BR', 'MEMS.PA', 'MERY.PA', 'MRK.PA', 'MRN.PA', 'MRNNV.PA', 'METEX.PA', 'MMT.PA', 'ML.PA', 'MLNV.PA', 'MUN.PA', 'MIKO.BR', 'MITRA.BR', 'NEDSE.AS', 'MONC.PA', 'MONT.BR', 'MOP.BR', 'MOPF.BR', 'MORE.AS', 'MOUR.BR', 'GREV.PA', 'NACON.PA', 'NANO.PA', 'KN.PA', 'NAVYA.PA', 'NEDAP.AS', 'NEOEN.PA', 'NRP.AS', 'NRO.PA', 'NSE.AS', 'NEWAY.AS', 'NEX.PA', 'NXI.PA', 'NEXTS.PA', 'COX.PA', 'NN.AS', 'NOKIA.PA', 'NOVI.AS', 'NR21.PA', 'NRG.PA', 'NSI.AS', 'NYR.BR', 'NYXH.BR', 'OCI.AS', 'SBT.PA', 'OLG.PA', 'ONTEX.BR', 'ORA.PA', 'OBEL.BR', 'ORANW.AS', 'ORAP.PA', 'KAZI.PA', 'ORDI.AS', 'OREGE.PA', 'ORP.PA', 'OSE.PA', 'OXUR.BR', 'PID.PA', 'PAR.PA', 'PARRO.PA', 'PSAT.PA', 'PAT.PA', 'PAY.BR', 'PCA.PA', 'RI.PA', 'PERR.PA', 'PSH.AS', 'PEUG.PA', 'PHA.PA', 'PHARM.AS', 'INPHI.AS', 'PHIA.AS', 'PIC.BR', 'VAC.PA', 'PVL.PA', 'POM.PA', 'PORF.AS', 'PNL.AS', 'POXEL.PA', 'PPLAA.AS', 'PPLAB.AS', 'PPLA.AS', 'PREC.PA', 'PROAC.PA', 'PWG.PA', 'PROL.PA', 'PRX.AS', 'PROX.BR', 'PSB.PA', 'PUB.PA', 'QRF.BR', 'QDT.PA', 'QFG.BR', 'RAL.PA', 'GDS.PA', 'RAND.AS', 'REC.BR', 'RX.PA', 'REINA.AS', 'REN.AS', 'RCO.PA', 'RNO.PA', 'RWI.AS', 'RES.BR', 'RET.BR', 'RXL.PA', 'RBT.PA', 'CBR.PA', 'CBE.PA', 'RBO.PA', 'ROOD.AS', 'ENGB.BR', 'ROTH.PA', 'ROU.BR', 'RDSA.AS', 'RDSB.AS', 'RUI.PA', 'SK.PA', 'SAF.PA', 'SGO.PA', 'SGONV.PA', 'SABE.PA', 'SAMS.PA', 'SAN.PA', 'SANNV.PA', 'DIM.PA', 'SAVE.PA', 'SBMO.AS', 'CBSM.PA', 'SCHD.BR', 'SLB.PA', 'SU.PA', 'SCR.PA', 'SCHP.PA', 'SGRO.PA', 'SLCO.PA', 'SELER.PA', 'SEQUA.BR', 'SEFER.PA', 'SESG.PA', 'SESL.PA', 'SRP.PA', 'SHUR.BR', 'SIFG.AS', 'GIRO.PA', 'LIGHT.AS', 'SII.PA', 'SIOE.BR', 'SIP.BR', 'SLIGR.AS', 'SMAR.BR', 'SMCP.PA', 'SMTPC.PA', 'SNOW.AS', 'SFCA.PA', 'GLE.PA', 'SW.PA', 'SEC.PA', 'SOF.BR', 'SOG.PA', 'SOI.PA', 'LOCAL.PA', 'S30.PA', 'SOLV.BR', 'SOLB.BR', 'SO.PA', 'SOP.PA', 'SPA.BR', 'SPIE.PA', 'SQI.PA', 'DPT.PA', 'STF.PA', 'STLA.PA', 'STRN.AS', 'STM.PA', 'SEV.PA', 'SWP.PA', 'SDG.PA', 'TKTT.PA', 'TAYN.PA', 'TCH.PA', 'TE.PA', 'FTI.PA', 'TNET.BR', 'TEP.PA', 'TVRB.PA', 'TERB.BR', 'TESB.BR', 'TES.PA', 'TFG.AS', 'TEXF.BR', 'TFI.PA', 'TFF.PA', 'HO.PA', 'HONV.PA', 'THEP.PA', 'TBIRD.AS', 'TIE.AS', 'TKO.PA', 'TINC.BR', 'TIPI.PA', 'TITC.BR', 'TWEKA.AS', 'TOM2.AS', 'FP.PA', 'EC.PA', 'TOUP.PA', 'EIFF.PA', 'TNG.PA', 'TRI.PA', 'TUB.BR', 'UBI.PA', 'UCB.BR', 'UMI.BR', 'URW.AS', 'UNBL.PA', 'UPG.BR', 'UNA.AS', 'UFF.PA', 'FPG.PA', 'VLK.AS', 'FR.PA', 'VK.PA', 'VLA.PA', 'VALUE.AS', 'PREVA.AS', 'VAN.BR', 'VASTN.AS', 'VASTB.BR', 'VIE.PA', 'VEON.AS', 'VRLA.PA', 'VMX.PA', 'VETO.PA', 'VGP.BR', 'VCT.PA', 'VDLO.PA', 'VIL.PA', 'RIN.PA', 'DG.PA', 'VIO.BR', 'VIRP.PA', 'SDT.PA', 'VIV.PA', 'VIVNV.PA', 'VVY.AS', 'VTA.AS', 'VLTSA.PA', 'VPK.AS', 'VRAP.PA', 'WEB.BR', 'WAVE.PA', 'WDP.BR', 'MF.PA', 'WHA.AS', 'WEHB.BR', 'WKL.AS', 'WLN.PA', 'XFAB.PA', 'XIL.PA', 'XIOR.BR', 'XPO.PA', 'YATRA.AS', 'CV.PA', 'ZENT.BR', '^FCHI', '^N100']

# CAC40
cac40 = ['EURUSD=X', '^FCHI', 'AI.PA', 'AIR.PA', 'ALO.PA', 'MT.PA', 'ATO.PA', 'CS.PA', 'BNP.PA', 'EN.PA', 'CAP.PA', 'CA.PA', 'ACA.PA', 'BN.PA', 'DSY.PA', 'ENGI.PA', 'EL.PA', 'RMS.PA', 'KER.PA', 'OR.PA', 'LR.PA', 'MC.PA', 'ML.PA', 'ORA.PA', 'RI.PA', 'PUB.PA', 'RNO.PA', 'SAF.PA', 'SGO.PA', 'SAN.PA', 'SU.PA', 'GLE.PA', 'STLA.PA', 'STM.PA', 'TEP.PA', 'HO.PA', 'FP.PA', 'URW.PA', 'VIE.PA', 'DG.PA', 'VIV.PA', 'WLN.PA']

# Selection of stocks & reference indices for testing
stocks_for_testing = ["EDF.PA", "ENGI.PA", "ABIO.PA", "ELEC.PA", "NEOEN.PA", "NSE.AS", "VLTSA.PA", "^FCHI", "^N100"]

small_test = ['SU.PA', 'VLTSA.PA']

# mystocks = euronext_pab
# mystocks = euronext_pab
mystocks = cac40
# mystocks = small_test

In [22]:
# ?yf.download

In [23]:
# Get all data
data = yf.download(
    tickers=mystocks,
    period="2y",
    interval="1d",
    threads=True,
)

[*********************100%***********************]  42 of 42 completed

2 Failed downloads:
- URW.PA: No data found, symbol may be delisted
- MT.PA: No data found for this date range, symbol may be delisted


In [24]:
# Pull closing prices and volumes. Check docs for all parameters
# data = yf.download(
#     tickers=mystocks,
#     period="2y",
#     interval="1d",
#     threads=True,
# )[["Adj Close", "Volume"]]

In [25]:
# Save data to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", data, "data.csv")

Writing 524 records to 'data.csv' in bucket 'euronext-stocks'.


In [26]:
# Store volumes in separate dataframe
df_volumes = data["Volume"]
df_volumes = df_volumes[mystocks]

In [27]:
df_volumes.shape

(524, 42)

In [28]:
df_volumes.tail()

,EURUSD=X,^FCHI,AI.PA,AIR.PA,ALO.PA,MT.PA,ATO.PA,CS.PA,BNP.PA,EN.PA,...,STLA.PA,STM.PA,TEP.PA,HO.PA,FP.PA,URW.PA,VIE.PA,DG.PA,VIV.PA,WLN.PA
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-17 00:00:00,0.0,62538700.0,572002.0,1087269.0,687944.0,NaN,492069.0,3375995.0,2510944.0,711134.0,...,3765451.0,1824844.0,108866.0,198511.0,4081153.0,NaN,1844052.0,703454.0,1924854.0,383247.0
2021-05-18 00:00:00,0.0,74129300.0,605892.0,1469378.0,963084.0,NaN,586574.0,4426932.0,2884081.0,933595.0,...,3313785.0,2195032.0,92992.0,208269.0,6041487.0,NaN,1250673.0,702687.0,3236112.0,369424.0
2021-05-19 00:00:00,0.0,92757600.0,836526.0,1947475.0,897552.0,NaN,758739.0,7370487.0,4155798.0,1763947.0,...,3416564.0,2959497.0,140683.0,356897.0,7239048.0,NaN,1231696.0,1185272.0,2003309.0,607858.0
2021-05-20 00:00:00,0.0,68528400.0,768895.0,1291926.0,1014223.0,NaN,496056.0,4297355.0,2960089.0,1485857.0,...,2591370.0,2802375.0,145993.0,246867.0,5630563.0,NaN,1326617.0,985247.0,1737001.0,421998.0
2021-05-21 00:00:00,0.0,75862000.0,748581.0,1454579.0,1062640.0,NaN,544415.0,5115531.0,4109887.0,1110099.0,...,4659128.0,2006560.0,110101.0,204694.0,5191318.0,NaN,1081473.0,862878.0,2557573.0,522588.0


In [29]:
# Save df_volumes to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", df_volumes, "df_volumes.csv")

Writing 524 records to 'df_volumes.csv' in bucket 'euronext-stocks'.


In [30]:
# Store prices in separate dataframe
df_prices = data["Adj Close"]
df_prices = df_prices[mystocks]

In [31]:
df_prices.shape

(524, 42)

In [32]:
df_prices.head()

,EURUSD=X,^FCHI,AI.PA,AIR.PA,ALO.PA,MT.PA,ATO.PA,CS.PA,BNP.PA,EN.PA,...,STLA.PA,STM.PA,TEP.PA,HO.PA,FP.PA,URW.PA,VIE.PA,DG.PA,VIV.PA,WLN.PA
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-21 00:00:00,1.116857,5385.459961,98.775711,114.512329,34.874493,NaN,69.750130,18.635651,37.346058,28.469492,...,NaN,14.475150,163.944016,95.904182,43.267834,NaN,19.207800,84.066292,23.960810,51.250000
2019-05-22 00:00:00,NaN,5378.979980,99.952133,114.473595,35.094494,NaN,71.618790,18.515982,36.943504,28.233629,...,NaN,14.274107,169.182465,95.432442,42.607059,NaN,19.293125,84.085098,23.863605,52.299999
2019-05-23 00:00:00,1.115548,5281.370117,98.035004,111.239433,34.909691,NaN,69.415741,18.284893,36.229702,27.692019,...,NaN,13.465029,166.757248,93.554932,41.202900,NaN,19.046627,83.144760,22.911007,50.099998
2019-05-24 00:00:00,1.118456,5316.509766,99.080704,112.846832,35.085693,NaN,69.828812,18.408689,36.441353,27.866732,...,NaN,13.396379,167.727341,93.517197,41.346363,NaN,19.331047,84.668121,22.940168,51.500000
2019-05-27 00:00:00,1.120963,5336.189941,99.734283,113.137321,35.499294,NaN,70.300896,18.470589,36.686207,27.814318,...,NaN,13.440511,169.279480,93.215271,41.602848,NaN,19.605986,85.044250,22.745760,51.500000


In [33]:
# Save df_prices to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", df_prices, "df_prices.csv")

Writing 524 records to 'df_prices.csv' in bucket 'euronext-stocks'.


### Get ticker metadata

In [34]:
mytickers = yf.Tickers(mystocks)

In [35]:
# Get metadata (inspect an example for all the types of metadata that can be captured)
ticker_info = {}
for ticker in list(mytickers.tickers):
    print(ticker)
    ticker_dets = []
    info = mytickers.tickers[ticker].info
    
    for md in metadata:
        try:
            ticker_dets.append(info[md])
        except:
            ticker_dets.append("Missing")
    ticker_info[ticker] = ticker_dets

EURUSD=X
^FCHI
AI.PA
AIR.PA
ALO.PA
MT.PA
ATO.PA
CS.PA
BNP.PA
EN.PA
CAP.PA
CA.PA
ACA.PA
BN.PA
DSY.PA
ENGI.PA
EL.PA
RMS.PA
KER.PA
OR.PA
LR.PA
MC.PA
ML.PA
ORA.PA
RI.PA
PUB.PA
RNO.PA
SAF.PA
SGO.PA
SAN.PA
SU.PA
GLE.PA
STLA.PA
STM.PA
TEP.PA
HO.PA
FP.PA
URW.PA
VIE.PA
DG.PA
VIV.PA
WLN.PA


In [36]:
# ticker_info

In [37]:
# Save ticker info to s3

s3 = boto3.client("s3")
serializedMyData = pickle.dumps(ticker_info)
# s3.put_object(Bucket="euronext-stocks", Key="ticker_info", Body=serializedMyData)
s3.put_object(Bucket="euronext-stocks", Key="ticker_info_cac40", Body=serializedMyData)
# s3.put_object(Bucket="euronext-stocks", Key="ticker_info_test", Body=serializedMyData)

{'ResponseMetadata': {'RequestId': '4D0BM2BS1VZ8F926',
  'HostId': '1QPKG+utYSJ8kQGE8PDnlXIvr0SKZNWygvBnFPDDEmMEMu5dLUvDp+kwrH5VCrM5TJlKuEX304M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1QPKG+utYSJ8kQGE8PDnlXIvr0SKZNWygvBnFPDDEmMEMu5dLUvDp+kwrH5VCrM5TJlKuEX304M=',
   'x-amz-request-id': '4D0BM2BS1VZ8F926',
   'date': 'Sun, 23 May 2021 08:17:49 GMT',
   'etag': '"4a442b3ae5427b928584d5f94b8d7d15"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4a442b3ae5427b928584d5f94b8d7d15"'}

In [38]:
# Get data required for ratio analysis

ratio_data = {}

for ticker in list(mytickers.tickers):
    print(ticker)
    ratio_dets = []
    info = mytickers.tickers[ticker].info
    cashflow_statement = mytickers.tickers[ticker].cashflow
    bs_statement = mytickers.tickers[ticker].balance_sheet
    pnl = mytickers.tickers[ticker].financials
    
    for sr in static_ratios:
        try:
            ratio_dets.append(info[sr])
        except:
            ratio_dets.append(np.nan)
    
    for cf in cashflow:
        try:
            ratio_dets.append(cashflow_statement[pd.to_datetime(prev_yr_end)].loc[cf])
        except:
            ratio_dets.append(np.nan)
    
    for bs in balsheet:
        try:
             ratio_dets.append(bs_statement[pd.to_datetime(prev_yr_end)].loc[bs])
        except:
            ratio_dets.append(np.nan)
    
    for fin in financials:
        try:
            ratio_dets.append(pnl[prev_yr_end].loc[fin])
        except:
            ratio_dets.append(np.nan)
    
    ratio_data[ticker] = ratio_dets

EURUSD=X
^FCHI
AI.PA
AIR.PA
ALO.PA
MT.PA
ATO.PA
CS.PA
BNP.PA
EN.PA
CAP.PA
CA.PA
ACA.PA
BN.PA
DSY.PA
ENGI.PA
EL.PA
RMS.PA
KER.PA
OR.PA
LR.PA
MC.PA
ML.PA
ORA.PA
RI.PA
PUB.PA
RNO.PA
SAF.PA
SGO.PA
SAN.PA
SU.PA
GLE.PA
STLA.PA
STM.PA
TEP.PA
HO.PA
FP.PA
URW.PA
VIE.PA
DG.PA
VIV.PA
WLN.PA


In [39]:
# store ratio data in dataframe
ratio_data_cols = static_ratios + cashflow + balsheet + financials # field names in correct order to be used in creating dataframe
df_ratio_data = pd.DataFrame(ratio_data, index=ratio_data_cols) 

In [40]:
df_ratio_data

,EURUSD=X,^FCHI,AI.PA,AIR.PA,ALO.PA,MT.PA,ATO.PA,CS.PA,BNP.PA,EN.PA,...,STLA.PA,STM.PA,TEP.PA,HO.PA,FP.PA,URW.PA,VIE.PA,DG.PA,VIV.PA,WLN.PA
profitMargins,NaN,NaN,1.188700e-01,-5.830000e-03,2.812000e-02,NaN,4.919000e-02,2.816000e-02,1.838800e-01,2.002000e-02,...,3.578000e-02,1.161400e-01,5.652000e-02,2.845000e-02,-3.283000e-02,NaN,5.960000e-03,2.815000e-02,8.949999e-02,5.957000e-02
payoutRatio,NaN,NaN,5.253000e-01,0.000000e+00,0.000000e+00,NaN,0.000000e+00,5.840000e-01,0.000000e+00,9.290000e-01,...,0.000000e+00,1.217000e-01,4.348000e-01,1.762000e-01,NaN,NaN,2.631600e+00,5.682000e-01,4.762000e-01,0.000000e+00
sharesOutstanding,NaN,NaN,4.719200e+08,7.855880e+08,3.712050e+08,NaN,1.092480e+08,2.386560e+09,1.248800e+09,3.807380e+08,...,3.124240e+09,8.989530e+08,5.871710e+07,2.129950e+08,2.629670e+09,NaN,5.672170e+08,5.669920e+08,1.086270e+09,2.792600e+08
trailingEps,NaN,NaN,5.140000e+00,-3.800000e-01,1.800000e+00,NaN,5.046000e+00,1.250000e+00,5.684000e+00,1.830000e+00,...,2.330000e+00,1.129000e+00,5.520000e+00,2.270000e+00,-1.350000e+00,NaN,2.750000e-01,2.200000e+00,1.260000e+00,8.140000e-01
forwardEps,NaN,NaN,5.750000e+00,5.580000e+00,NaN,NaN,9.620000e+00,2.800000e+00,6.110000e+00,2.930000e+00,...,NaN,1.340000e+00,8.010000e+00,6.360000e+00,5.970000e+00,NaN,NaN,5.830000e+00,1.050000e+00,1.170000e+00
Total Cash From Operating Activities,NaN,NaN,5.205700e+09,-5.420000e+09,NaN,NaN,1.219000e+09,2.553400e+10,1.393240e+11,3.406000e+09,...,6.202000e+09,2.093000e+09,9.890000e+08,1.345600e+09,1.480300e+10,NaN,2.725000e+09,6.675000e+09,1.226000e+09,5.113000e+08
Total Liab,NaN,NaN,2.297250e+10,1.036390e+11,NaN,NaN,1.125600e+10,7.191930e+11,2.371142e+12,2.882000e+10,...,4.963600e+10,5.948000e+09,4.615000e+09,2.649790e+10,1.600470e+11,NaN,3.704920e+10,6.814100e+10,2.169100e+10,9.283600e+09
Total Stockholder Equity,NaN,NaN,1.854230e+10,6.445000e+09,NaN,NaN,6.861000e+09,7.161000e+10,1.127990e+11,1.034000e+10,...,2.129400e+10,8.448000e+09,2.409000e+09,5.114900e+09,1.037020e+11,NaN,7.216200e+09,2.086300e+10,1.575900e+10,9.480600e+09
Total Current Assets,NaN,NaN,6.366300e+09,5.840000e+10,NaN,NaN,7.816000e+09,1.268810e+11,1.165380e+12,1.921400e+10,...,3.525800e+10,7.610000e+09,2.680000e+09,1.742650e+10,7.967900e+10,NaN,1.762030e+10,3.203900e+10,8.433000e+09,4.606100e+09
Total Current Liabilities,NaN,NaN,7.275100e+09,4.977100e+10,NaN,NaN,5.871000e+09,2.131180e+11,1.703883e+12,1.938400e+10,...,3.081300e+10,3.053000e+09,1.641000e+09,1.717570e+10,6.467600e+10,NaN,2.044960e+10,3.346800e+10,1.330800e+10,4.523100e+09


In [41]:
# Save df_ratio_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_ratio_data, "df_ratio_data.csv")

Writing 11 records to 'df_ratio_data.csv' in bucket 'euronext-stocks'.


### Pre-compute ratios and store to s3

#### Trailing PE

In [42]:
# Table of Trailing P/E Ratios
trailing_pe_data = {}
for ticker in df_prices.columns:
    trailing_pe_data[ticker] = df_prices[ticker] / df_ratio_data[ticker].loc["trailingEps"]
df_trailing_pe_data = pd.DataFrame(trailing_pe_data)

In [43]:
# Save df_trailing_pe_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_trailing_pe_data, "df_trailing_pe_data.csv")

Writing 524 records to 'df_trailing_pe_data.csv' in bucket 'euronext-stocks'.


#### Forward PE

In [44]:
# Table of Forward P/E Ratios
forward_pe_data = {}
for ticker in df_prices.columns:
    forward_pe_data[ticker] = df_prices[ticker] / df_ratio_data[ticker].loc["forwardEps"]
df_forward_pe_data = pd.DataFrame(forward_pe_data)

In [45]:
# Save df_forward_pe_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_forward_pe_data, "df_forward_pe_data.csv")

Writing 524 records to 'df_forward_pe_data.csv' in bucket 'euronext-stocks'.


#### Price to Sales

In [46]:
# Table of Price to Sales Ratios
p2s_data = {}
for ticker in df_prices.columns:
    p2s_data[ticker] = df_prices[ticker] / (df_ratio_data[ticker].loc["Total Revenue"] / df_ratio_data[ticker].loc["sharesOutstanding"])
df_p2s_data = pd.DataFrame(p2s_data)

In [47]:
# Save df_p2s_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_p2s_data, "df_p2s_data.csv")

Writing 524 records to 'df_p2s_data.csv' in bucket 'euronext-stocks'.


#### Profit Margin

In [48]:
su = yf.Ticker("SU.PA")
# su.financials # [pd.to_datetime('2020-12-31')].loc["Total Liab"]
su.financials.loc["Net Income Applicable To Common Shares"] /su.financials.loc["Total Revenue"]


2020-12-31    0.0845026
2019-12-31    0.0888504
2018-12-31    0.0907465
2017-12-31    0.0868933
dtype: object

In [49]:
# Table of Profit Margins
pm_data = {}
for ticker in df_prices.columns:
    dets = yf.Ticker(ticker)
    try:
        rev = dets.financials.loc["Total Revenue"]
    except:
        rev = np.nan
    try:
        profit = dets.financials.loc["Net Income Applicable To Common Shares"]
    except:
        profit = np.nan
    pm = profit/rev
    pm_data[ticker] = pm
df_pm_data = pd.DataFrame(pm_data)

In [50]:
# Save df_pm_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_pm_data, "df_pm_data.csv")

Writing 15 records to 'df_pm_data.csv' in bucket 'euronext-stocks'.


#### Dividend payout ratio

In [51]:
# Table of Dividend payout ratios
dpr_data = {}
for ticker in df_prices.columns:
    dpr_data[ticker] = df_ratio_data[ticker].loc["payoutRatio"]
series_dpr_data = pd.Series(dpr_data)

In [52]:
# Plot dividend payout ratios
# series_dpr_data.sort_values().plot.barh(figsize=(12,10))
# plt.grid()

In [53]:
# Save series_dpr_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", series_dpr_data, "series_dpr_data.csv")

Writing 42 records to 'series_dpr_data.csv' in bucket 'euronext-stocks'.


#### Price to Free Cashflow ratio

In [54]:
# Table of Price to Free Cashflow ratios
pfcf_data = {}
for ticker in df_prices.columns:
#     print(ticker)
    dets = yf.Ticker(ticker)
    try:
        coa = dets.cashflow[prev_yr_end].loc["Total Cash From Operating Activities"]
        capex = dets.cashflow[prev_yr_end].loc["Capital Expenditures"]
        ocf = coa + capex 
    except:
        ocf = np.nan
#     print(ocf)
    sos = df_ratio_data[ticker].loc["sharesOutstanding"]
#     print(sos)
    ocfs = ocf / sos
#     print(ocfs)
#     print(df_prices[ticker])
    pfcf_data[ticker] = df_prices[ticker] / ocfs
df_pfcf_data = pd.DataFrame(pfcf_data)

In [55]:
# Save df_pfcf_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_pfcf_data, "df_pfcf_data.csv")

Writing 524 records to 'df_pfcf_data.csv' in bucket 'euronext-stocks'.


#### Debt to Equity ratio

In [80]:
# Table of Debt Equity ratios
de_data = {}
for ticker in df_prices.columns:
#     print(ticker)
    dets = yf.Ticker(ticker)
    try:
        tot_liab = dets.balance_sheet.loc["Total Liab"]
    except:
        tot_liab = np.nan
#     print(tot_liab)
    try:
        tot_se = dets.balance_sheet.loc["Total Stockholder Equity"]
    except:
        tot_se = np.nan
#     print(tot_se)
    de_ratio = tot_liab/tot_se
#     print(de_ratio)
    de_data[ticker] = de_ratio
df_de_data = pd.DataFrame(de_data)

In [94]:
# Save df_de_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_de_data, "df_de_data.csv")

Writing 15 records to 'df_de_data.csv' in bucket 'euronext-stocks'.


#### Current ratio

In [90]:
# Table of Current ratios
cr_data = {}
for ticker in df_prices.columns:
#     print(ticker)
    dets = yf.Ticker(ticker)
    try:
        ca = dets.balance_sheet.loc["Total Current Assets"]
    except:
        ca = np.nan
#     print(ca)
    try:
        cl = dets.balance_sheet.loc["Total Current Liabilities"]
    except:
        cl = np.nan
#     print(cl)
    current_ratio = ca/cl
#     print(current_ratio)
    cr_data[ticker] = current_ratio
df_cr_data = pd.DataFrame(cr_data)

In [95]:
# Save df_cr_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_cr_data, "df_cr_data.csv")

Writing 15 records to 'df_cr_data.csv' in bucket 'euronext-stocks'.
